[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamindb/blob/main/docs/bio-registries.ipynb)

# Manage biological registries 

Registries can anchor dry & wetlab work in a common framework to more easily access & model data.

In addition of being standard relational database tables, LaminDB allows you to leverage public ontologies for biological registries, based on plug-in {mod}`lnschema_bionty`. With this, you can create records from and validate against external sources.

## Setup

Let us create an instance that has {mod}`lnschema_bionty` mounted:

In [ ]:
!lamin init --storage ./test-registries --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

Let's pre-populate the cell type registry with a few records:

In [ ]:
lb.Species.from_bionty(name="human").save()
lb.CellType.from_bionty(name="T cell").save()
lb.CellType(name="my T cell subtype").save()

## Search or look up terms from a public ontology

Let's start with a public ontology for cell types.

[Bionty](https://lamin.ai/docs/bionty/bionty.bionty) - short for "biological entity" - is a class for accessing ontologies and similar resources:

In [ ]:
bionty = lb.CellType.bionty()

In [ ]:
bionty

We can use it to search cell types:

In [ ]:
bionty.search("gamma delta T cell").head(3)

And we can also use it to look up cell types with auto-complete:

In [ ]:
lookup = bionty.lookup()
lookup.gamma_delta_t_cell

## Create a record for an in-house registry

Create a registry record directly by passing the result of a Bionty lookup:

In [ ]:
lb.CellType(lookup.gamma_delta_t_cell)

Or specify to create from Bionty public source:

In [ ]:
gdt_cell = lb.CellType.from_bionty(ontology_id=lookup.gamma_delta_t_cell.ontology_id)

In [ ]:
gdt_cell

Records creation from Bionty is synonyms aware:

In [ ]:
lb.CellType.from_bionty(name="B lymphocyte")

When we save this record to the registry, logging informs us that we're also saving parent ontological terms.


```{dropdown} Will I always see a ton of parents being saved?

No, this only happens a single time.

- If we accidentally save the same record again, lamindb will recognize that the record and all parents are already in the registry.
- If we save another record that has overlapping parents, only new parents will be saved.

```

In [ ]:
gdt_cell.save()

View the ontological hierarchy:

In [ ]:
gdt_cell.view_parents()

Or access the parents directly:

In [ ]:
gdt_cell.parents.df()

You can construct custom hierarchies of terms by specifying parents:

In [ ]:
my_celltype = lb.CellType.filter(name="my T cell subtype").one()
my_celltype.parents.add(gdt_cell)

In [ ]:
gdt_cell.view_parents(distance=2, with_children=True)

This cell type and all its parents can now be queried & searched in the registry using `lb.CellType.filter` and `lb.CellType.search`.

Further down the guide, we'll see how this will help us to annotate and validate files & datasets!

## Bulk create records by parsing data

Consider a DataFrame-based example:

In [ ]:
adata = ln.dev.datasets.anndata_with_obs()

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.cell_type.value_counts()

You need to specify a field correspond to the values you are passing, for instance "CellType.name" or "CellType.ontology_id" in this case.

It's important to note that `Registry.from_values()` does _not_ create records that are non-validated against in-house registry or public Bionty sources.

`Registry.from_values()` creates records in the following steps:

1. If existing DB records that match the input field values, return records without creating new
2. If input values matches synonyms associated with existing DB records, return records without creating new
3. (`lnschema_bionty` only) For non-existing DB records, create records from Bionty that matches corresponding Bionty field
4. (`lnschema_bionty` only) Create records from Bionty that matches synonyms

No records will be created if input field values that are not mappable in the above ways.

In [ ]:
# Input has 4 unique values of cell type names
adata.obs.cell_type.unique().tolist()

In [ ]:
cell_types = lb.CellType.from_values(adata.obs.cell_type)

cell_types

What if the input contains synonyms? They are recognized:

In [ ]:
celltype_names = [
    "gamma-delta T cell",  # existing record with the same name
    "T lymphocyte",  # existing record with synonym
    "hepatocyte",  # Bionty record with the same name
    "HSC",  # Bionty record with synonym
    "my new cell type",  # Not exist in DB, not exist in Bionty
]

In [ ]:
lb.CellType.from_values(celltype_names)

Note that no record is created from "my new cell type". If you are sure to register it, use the default constructor:

In [ ]:
my_celltype = lb.CellType(name="my new cell type")
my_celltype.save()

Similarly, we can create entries based on cell type ontology ids that eliminates the synonyms ambiguity:

In [ ]:
# Input has 3 unique values and 1 empty string (empty values don't result a record)
adata.obs.cell_type_id.unique().tolist()

In [ ]:
lb.CellType.from_values(adata.obs.cell_type_id, lb.CellType.ontology_id)

If we're happy with `cell_types` records (in particular, are sure that "my new cell type" needs to be added to the DB), we save them to the DB in one transaction:

In [ ]:
ln.save(cell_types)

Now let's check out our in-house registry:

In [ ]:
lb.CellType.filter().df()

## Search or lookup terms in the DB

In [ ]:
lb.CellType.search("gamma delta T cell").head(2)

In [ ]:
celltype_db_lookup = lb.CellType.lookup()

In [ ]:
hsc_record = celltype_db_lookup.hematopoietic_stem_cell

In [ ]:
hsc_record

## Map or add synonyms to terms in the DB

```{important}

While records creation via `Registry.from_values()` is synonyms aware; `.validate()` is not.

In order to pass validation, run `.standardize()` so that only validated terms are associated with your data.
```

In [ ]:
# synonyms aware
lb.CellType.from_values(["HSC", "blood forming stem cell"])

In [ ]:
# synonyms are not validated
lb.CellType.validate(["HSC", "blood forming stem cell"]);

Convert synonyms to standardized names:

In [ ]:
lb.CellType.standardize(["HSC", "blood forming stem cell"])

Add a new synonym to a record:

In [ ]:
hsc_record.add_synonym("HSCs")

Now this new synonym can also be mapped:

In [ ]:
lb.CellType.standardize(["HSCs"])

A special synonym is "abbr" (abbreviation), which has its own field and can be assigned via:

In [ ]:
hsc_record.set_abbr("HSC")

Similarly, users can create a lookup object from abbr field:

In [ ]:
celltype_db_lookup = lb.CellType.lookup("abbr")
hsc_record = celltype_db_lookup.hsc
hsc_record

The same workflow works for all of `lnschema_bionty`'s ORMs.

## Multi-species registries

Multi-species ORMs are species aware, for instance, Gene:

In [ ]:
lb.Gene.from_bionty(
    symbol="TCF7", species="human"  # error is raised without passing species
)

You can also omit the `species` argument, if you configure it globally:

In [ ]:
lb.settings.species = "mouse"

In [ ]:
lb.Gene.from_bionty(symbol="Ap5b1")

## Track underlying ontology sources

Under-the-hood, ontology sources are automatically tracked:

In [ ]:
lb.BiontySource.filter(currently_used=True).df()

Each record is linked to a versioned bionty source (if it was created from bionty):

In [ ]:
cell_type_record = lb.CellType.filter(name="hepatocyte").one()
cell_type_record.bionty_source

In [ ]:
!lamin delete --force test-registries
!rm -r test-registries